In [22]:
import json
import requests
import itertools
from collections import Counter, OrderedDict
import pandas as pd
%matplotlib inline
from tqdm import tqdm

In [23]:
file_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/PCSE/OTAR-GP-DS-Data.csv'
annot_csv = pd.read_csv(file_path, names=['pmc_id', 'section', 'sentence', 'EP_GP', 'ML_GP', 'EP_DS', 'ML_DS'],sep='\t')

In [3]:
annot_csv.columns

Index(['pmc_id', 'section', 'sentence', 'EP_GP', 'ML_GP', 'EP_DS', 'ML_DS'], dtype='object')

In [4]:
len(annot_csv)

1214918

In [5]:
annot_csv_ = annot_csv.head(10)
annot_csv_

,pmc_id,section,sentence,EP_GP,ML_GP,EP_DS,ML_DS
0,abstract/MED/1002003,abstract,The plasma levels of the cold-insoluble globul...,"['cold-insoluble globulin', 'cold-insoluble gl...",['globulin'],"['inflammations', 'chronic lymphatic leukemias...","['chronic lymphatic leukemias', 'rheumatic dis..."
1,abstract/MED/1002003,title,Factor-VIII-related antigen and cold-insoluble...,"['cold-insoluble globulin', 'cold-insoluble gl...",['globulin'],"['leukemias', 'carcinomas']","['leukemias', 'carcinomas']"
2,abstract/MED/1002008,abstract,"The whole blood clotting time, plasma fibrinog...",['coagulation factors II'],"['fibrinogen', 'coagulation factors II']",['acute myocardial infarction'],['acute myocardial infarction']
3,abstract/MED/1002112,title,Hyperglycemia induced by somatostatin in norma...,['somatostatin'],['somatostatin'],['Hyperglycemia'],['Hyperglycemia']
4,abstract/MED/1002113,abstract,"Concentrations of testosterone, delta4-androst...",['delta4'],[],['hirsutism'],"['hirsutism', 'menstrual disorders']"
5,abstract/MED/10021292,abstract,Irinotecan (CPT-11) is a DNA topoisomerase I i...,"['DNA topoisomerase I', 'DNA topoisomerase I']",['DNA topoisomerase I'],"['cancer', 'tumors']","['cancer', 'solid tumors']"
6,abstract/MED/10021296,abstract,Paclitaxel-resistant subclones (T30 and T30-Re...,['Mes'],[],['uterine sarcoma'],['uterine sarcoma']
7,abstract/MED/10021299,abstract,This implies that these two parameters may be ...,['WAF1'],['WAF1'],['endometrial carcinomas'],['endometrial carcinomas']
8,abstract/MED/10021299,title,WAF1 genotype and endometrial cancer susceptib...,['WAF1'],['WAF1'],['cancer'],['endometrial cancer']
9,abstract/MED/10021305,abstract,"In invasive vulvar carcinoma, high VEGF expres...","['VEGF', 'MVD']",['VEGF'],"['vulvar carcinoma', 'vulvar carcinoma']","['invasive vulvar carcinoma', 'MVD']"


# Analysis

In [24]:
from ast import literal_eval
from tqdm import tqdm

# Remove duplicate entries of the GP from the CSV
# 'pmc_id', 'section', 'sentence', 'EP-GP', 'ML-GP', 'EP-DS', 'ML-DS'
updated_GP_ner_non_duplicates = []
updated_DS_ner_non_duplicates = []

updated_ML_DS_ner= []
updated_ML_GP_ner= []

for index_, each_annotation in tqdm(enumerate(annot_csv.itertuples()),total=len(annot_csv)):
    GP_ner = literal_eval(each_annotation.EP_GP)
    DS_ner = literal_eval(each_annotation.EP_DS)
    
    if literal_eval(each_annotation.ML_GP):
        updated_ML_GP_ner.append(literal_eval(each_annotation.ML_GP))
    else:
        updated_ML_GP_ner.append([''])
                    
    if literal_eval(each_annotation.ML_DS):
        updated_ML_DS_ner.append(literal_eval(each_annotation.ML_DS))
    else:
        updated_ML_DS_ner.append(['']) 
                    
                    
    
    otar_sentence = each_annotation.sentence
    new_GP_ner = []
    new_DS_ner = []
    
    for each_epmc_ner in set(GP_ner):
        if GP_ner.count(each_epmc_ner) > 1:
            count = otar_sentence.count(each_epmc_ner)
            new_GP_ner.extend([each_epmc_ner]*count)
        else:
            new_GP_ner.append(each_epmc_ner)
    updated_GP_ner_non_duplicates.append(new_GP_ner)    
            
    for each_epmc_ner in set(DS_ner):
        if DS_ner.count(each_epmc_ner) > 1:
            count = otar_sentence.count(each_epmc_ner)
            new_DS_ner.extend([each_epmc_ner]*count)
        else:
            new_DS_ner.append(each_epmc_ner)
#         print(new_epmc_ner)
    updated_DS_ner_non_duplicates.append(new_DS_ner)

100%|██████████| 1334594/1334594 [01:28<00:00, 15165.56it/s]


In [25]:
len(updated_ML_GP_ner), len(updated_DS_ner_non_duplicates)

(1334594, 1334594)

In [26]:
annot_csv['EP_GP'] =  updated_GP_ner_non_duplicates
annot_csv['EP_DS'] =  updated_DS_ner_non_duplicates

annot_csv['ML_GP'] =  updated_ML_GP_ner
annot_csv['ML_DS'] =  updated_ML_DS_ner

annot_csv.head(10)

,pmc_id,section,sentence,EP_GP,ML_GP,EP_DS,ML_DS
0,abstract/MED/1002003,abstract,The plasma levels of the cold-insoluble globul...,[cold-insoluble globulin],[globulin],"[chronic lymphatic leukemias, rheumatic diseas...","[chronic lymphatic leukemias, rheumatic diseases]"
1,abstract/MED/1002003,title,Factor-VIII-related antigen and cold-insoluble...,[cold-insoluble globulin],[globulin],"[leukemias, carcinomas]","[leukemias, carcinomas]"
2,abstract/MED/1002008,abstract,"The whole blood clotting time, plasma fibrinog...",[coagulation factors II],"[fibrinogen, coagulation factors II]",[acute myocardial infarction],[acute myocardial infarction]
3,abstract/MED/1002112,title,Hyperglycemia induced by somatostatin in norma...,[somatostatin],[somatostatin],[Hyperglycemia],[Hyperglycemia]
4,abstract/MED/1002113,abstract,"Concentrations of testosterone, delta4-androst...",[delta4],[],[hirsutism],"[hirsutism, menstrual disorders]"
5,abstract/MED/10021292,abstract,Irinotecan (CPT-11) is a DNA topoisomerase I i...,[DNA topoisomerase I],[DNA topoisomerase I],"[tumors, cancer]","[cancer, solid tumors]"
6,abstract/MED/10021296,abstract,Paclitaxel-resistant subclones (T30 and T30-Re...,[Mes],[],[uterine sarcoma],[uterine sarcoma]
7,abstract/MED/10021299,abstract,This implies that these two parameters may be ...,[WAF1],[WAF1],[endometrial carcinomas],[endometrial carcinomas]
8,abstract/MED/10021299,title,WAF1 genotype and endometrial cancer susceptib...,[WAF1],[WAF1],[cancer],[endometrial cancer]
9,abstract/MED/10021305,abstract,"In invasive vulvar carcinoma, high VEGF expres...","[VEGF, MVD]",[VEGF],[vulvar carcinoma],"[invasive vulvar carcinoma, MVD]"


In [9]:
k_exacts = []
k_preds = []
k_missing = []


for index_, each_annotation in tqdm(enumerate(annot_csv.itertuples()),total=len(annot_csv)):
#     if index_ <10:
    #     pmc_id = each_annotation.pmc_id
    #     section = each_annotation.section
    #     epmc_sentence = each_annotation.sentence
#         k_exacts.append(literal_eval(each_annotation.eurpmc_ner))
        k_exacts.append(each_annotation.EP_GP)
        k_preds.append(each_annotation.ML_GP)

    

100%|██████████| 1214918/1214918 [00:02<00:00, 474091.98it/s]


In [10]:
def no_intersection(lst1, lst2): 
    miss = []
    temp = set(lst2) 
    for value in lst1:
        scores = [fuzz.partial_ratio(value,sublist) for sublist in temp]
        if 100 not in scores:
#         if any(i < 70for i in scores):
            miss.append(value)  
    return miss
    

In [11]:
from fuzzywuzzy import fuzz

In [12]:
k_missing = []
for i,j in tqdm(zip(k_exacts,k_preds), total = len(k_exacts)):
    if i:
        missing = no_intersection(i, j)
        if missing:
            k_missing.append(missing)

100%|██████████| 1214918/1214918 [00:29<00:00, 41686.12it/s]


In [13]:
missing_data = list(itertools.chain.from_iterable(k_missing))
count = Counter(missing_data)
y = OrderedDict(count.most_common())

In [14]:
import pickle

path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/PCSE_Analysis/'
with open(path+'k_exacts', 'wb') as fp:
    pickle.dump(k_exacts, fp)
    
with open(path+'k_preds', 'wb') as fp:
    pickle.dump(k_preds, fp) 

In [15]:
anno_data = list(itertools.chain.from_iterable(k_exacts))
anno_count = Counter(anno_data)
anno_y = OrderedDict(anno_count.most_common())

In [16]:
import csv

with open(path+"OTAR_frequencies_GP_20K.csv", "w",  newline='\n') as outfile:
    public_writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    for keys, values in y.items(): # Get keys from the missing
        filt_per = (values/anno_y[keys])*100
        new_row = [keys, str(values), str(anno_y[keys]), str(filt_per)]
        public_writer.writerows([new_row])

In [17]:
## Convert to json

In [18]:
jsonFilePath = 'OTAR_dump.json'
annot_csv.to_json(path+jsonFilePath, orient='records')

In [19]:
path+"OTAR_frequencies_GP_20K.csv"

'/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/PCSE_Analysis/OTAR_frequencies_GP_20K.csv'

In [20]:
# convert annotations freq to json
columns_freq = ['Entity', 'Freq_of_ML_removed','EPMC_total_Freq','Percentage_removed']
anno_freq_df = pd.read_csv(path+"OTAR_frequencies_GP_20K.csv", sep = '\t', error_bad_lines=False, names = columns_freq)

In [21]:
jsonFilePath = 'OTAR_freqs.json'
anno_freq_df.to_json(path+jsonFilePath, orient='records')

In [27]:
# Disease Freqs

k_exacts = []
k_preds = []
k_missing = []


for index_, each_annotation in tqdm(enumerate(annot_csv.itertuples()),total=len(annot_csv)):
#     if index_ <10:
    #     pmc_id = each_annotation.pmc_id
    #     section = each_annotation.section
    #     epmc_sentence = each_annotation.sentence
#         k_exacts.append(literal_eval(each_annotation.eurpmc_ner))
        k_exacts.append(each_annotation.EP_DS)
        k_preds.append(each_annotation.ML_DS)

        
k_missing = []
for i,j in tqdm(zip(k_exacts,k_preds), total = len(k_exacts)):
    if i:
        missing = no_intersection(i, j)
        if missing:
            k_missing.append(missing) 
            
            
missing_data = list(itertools.chain.from_iterable(k_missing))
count = Counter(missing_data)
y = OrderedDict(count.most_common())


import pickle

path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/PCSE_Analysis/'
with open(path+'k_exacts_DS', 'wb') as fp:
    pickle.dump(k_exacts, fp)
    
with open(path+'k_preds_DS', 'wb') as fp:
    pickle.dump(k_preds, fp) 
    


100%|██████████| 1334594/1334594 [00:21<00:00, 60767.89it/s]


KeyError: 'inflammation'

In [29]:
anno_data = list(itertools.chain.from_iterable(k_exacts))
anno_count = Counter(anno_data)
anno_y = OrderedDict(anno_count.most_common())

import csv

with open(path+"OTAR_frequencies_DS_20K.csv", "w",  newline='\n') as outfile:
    public_writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    for keys, values in y.items(): # Get keys from the missing
        try:
            filt_per = (values/anno_y[keys])*100
            new_row = [keys, str(values), str(anno_y[keys]), str(filt_per)]
            public_writer.writerows([new_row]) 
        except:
            pass
        
        
        
path+"OTAR_frequencies_DS_20K.csv" 


# convert annotations freq to json
columns_freq = ['Entity', 'Freq_of_ML_removed','EPMC_total_Freq','Percentage_removed']
anno_freq_df = pd.read_csv(path+"OTAR_frequencies_DS_20K.csv", sep = '\t', error_bad_lines=False, names = columns_freq)



jsonFilePath = 'OTAR_DS_freqs.json'
anno_freq_df.to_json(path+jsonFilePath, orient='records')